In [4]:
pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/99/a1/5219921a9bcb5388623f81908ff43d606aec2e512e0c02f6eb49f3b3caa8/xgboost-2.1.1-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [8]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/Users/tarunreddy/Desktop/Projects/Mushroom_Kaggle/clean_mush.csv')
df.shape

(3111859, 118)

In [10]:
# Split the data
X = df.iloc[:, 1:]  # Features (columns 2 to 118)
y = df.iloc[:, 0]   # Target (first column)

# Check the shapes to confirm the split
print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: (3111859, 117)
Target shape: (3111859,)


In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Check the shapes of the splits
print("Training Features shape:", X_train.shape)
print("Validation Features shape:", X_val.shape)
print("Training Target shape:", y_train.shape)
print("Validation Target shape:", y_val.shape)

Training Features shape: (2489487, 117)
Validation Features shape: (622372, 117)
Training Target shape: (2489487,)
Validation Target shape: (622372,)


In [20]:
param_grid = {
    'max_depth': [3, 5, 7, 9, 11],
    'eta': [0.3],
    'objective': ['binary:logistic'],
    'eval_metric': ['logloss'],
    'seed': [42]
}

In [22]:
xgb_model = xgb.XGBClassifier(n_estimators=100, use_label_encoder=False)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

In [24]:
# Fit the model with the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print(f"Best max_depth: {grid_search.best_params_['max_depth']}")
print(f"Best score: {grid_search.best_score_}")

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/Users/tarunreddy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:04:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tarunreddy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:05:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tarunreddy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:05:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tarunreddy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:05:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, U

Best max_depth: 11
Best score: 0.9919409099143719


In [26]:
# Get the best model
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_val)

# Calculate accuracy on the test set
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")


Accuracy on the test set: 99.21%


## Testing

In [29]:
# Load the CSV file into a DataFrame
df_test = pd.read_csv('/Users/tarunreddy/Desktop/Projects/Mushroom_Kaggle/clean_mush_test.csv')
df_test.shape

(2077964, 118)

In [31]:
X_test = df_test.iloc[:, 1:]  # Features (columns 2 to 118)
id = df_test.iloc[:, 0]   # Target (first column)

# Check the shapes to confirm the split
print("Features shape:", X_test.shape)
print("Target shape:", id.shape)

Features shape: (2077964, 117)
Target shape: (2077964,)


In [33]:
# Check if column names and their order are the same
columns_are_equal = (X.columns.tolist() == X_test.columns.tolist())

# Check if datatypes are the same
dtypes_are_equal = (X.dtypes.tolist() == X_test.dtypes.tolist())

if columns_are_equal and dtypes_are_equal:
    print("Both DataFrames have the same columns in the same order and the same datatypes.")
else:
    if not columns_are_equal:
        print("The DataFrames do not have the same columns in the same order.")
    if not dtypes_are_equal:
        print("The DataFrames do not have the same datatypes.")


Both DataFrames have the same columns in the same order and the same datatypes.


In [35]:
# Make predictions
predictions = best_model.predict(X_test)

In [87]:
# Convert predictions to class labels ('p' or 'e')
class_labels = ['p' if pred == 1 else 'e' for pred in predictions]
# Create a new DataFrame with the id and class labels
results_df = pd.DataFrame({
    'id': id,
    'class': class_labels
})

# Save the DataFrame to a CSV file
results_df.to_csv('predictions_XG.csv', index=False)

print("CSV file with predictions created successfully.")

CSV file with predictions created successfully.


In [89]:
# Get unique values and their counts
value_counts = results_df['class'].value_counts()

print("Unique values and counts:")
print(value_counts)

Unique values and counts:
class
p    1133533
e     944431
Name: count, dtype: int64


In [91]:
best_model.save_model('best_xg_model.xgb')

/Users/tarunreddy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:02:25] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
